## Self borrowing activity on Compound V2

This notebook intends to illustrate the correlation between self-borrowing and the incentives intern to the protocol. 

In [37]:
import altair as alt
alt.data_transformers.disable_max_rows()
import pandas as pd
from datetime import datetime

In [38]:
liability_matrix = pd.read_csv('../data/balance_sheets/daily_liability_matrix.csv')
liability_matrix = liability_matrix.rename(columns={'date': 'timestamp'})
liability_matrix['timestamp'] = pd.to_datetime(liability_matrix['timestamp'])
liability_matrix

,symbol_bor,symbol_col,account,matchBorrowUSD,matchCollateralUSD,effectiveUSD,timestamp
0,cBAT,cBAT,1,1.028105e+01,1.763471e+02,1.028105e+01,2019-05-07
1,cBAT,cETH,2,1.895208e+01,3.152674e+02,1.895208e+01,2019-05-07
2,cBAT,cREP,1,2.004187e+00,3.437709e+01,2.004187e+00,2019-05-07
3,cBAT,cSAI,1,2.686493e+01,4.608043e+02,2.686493e+01,2019-05-07
4,cBAT,cUSDC,1,8.139770e+00,1.396185e+02,8.139770e+00,2019-05-07
...,...,...,...,...,...,...,...
209063,cZRX,cUSDC,48,1.068964e+05,1.503970e+06,1.068959e+05,2023-07-31
209064,cZRX,cWBTC,15,1.554430e+02,4.561121e+02,1.547590e+02,2023-07-31
209065,cZRX,cWBTC2,12,1.970710e+02,3.269187e+02,1.953021e+02,2023-07-31
209066,cZRX,cYFI,1,8.481281e-10,1.729400e-05,8.481281e-10,2023-07-31


In [39]:
self_borrow = liability_matrix[liability_matrix['symbol_bor'] == liability_matrix['symbol_col']].reset_index(drop=True)
self_borrow = self_borrow.groupby(['symbol_bor', 'timestamp']).sum().drop(['symbol_col', 'account', 'effectiveUSD', 'matchCollateralUSD'], axis=1).reset_index()
self_borrow = self_borrow.rename(columns={'matchBorrowUSD': 'selfBorrowUSD'})
self_borrow

,symbol_bor,timestamp,selfBorrowUSD
0,cAAVE,2021-08-19,2489.868402
1,cAAVE,2021-08-20,2722.808655
2,cAAVE,2021-08-21,4610.444347
3,cAAVE,2021-08-22,4503.303663
4,cAAVE,2021-08-23,4627.488659
...,...,...,...
14872,cZRX,2023-07-27,52935.397436
14873,cZRX,2023-07-28,52933.375206
14874,cZRX,2023-07-29,52931.189305
14875,cZRX,2023-07-30,52933.307208


In [40]:
total_borrow = liability_matrix.groupby('timestamp').sum().drop(['symbol_col', 'account', 'effectiveUSD', 'symbol_bor'], axis=1).reset_index()
total_borrow = total_borrow.rename(columns={'matchBorrowUSD': 'totalBorrowUSD', 'matchCollateralUSD': 'totalCollateralUSD'})
total_borrow

,timestamp,totalBorrowUSD,totalCollateralUSD
0,2019-05-07,5.869795e+02,1.005838e+04
1,2019-05-08,4.718694e+03,1.258688e+04
2,2019-05-09,5.818155e+03,1.258489e+04
3,2019-05-10,5.852302e+03,1.158306e+04
4,2019-05-11,6.179203e+03,1.246662e+04
...,...,...,...
1542,2023-07-27,5.696505e+08,1.259088e+09
1543,2023-07-28,5.718996e+08,1.268614e+09
1544,2023-07-29,5.725436e+08,1.268509e+09
1545,2023-07-30,5.577766e+08,1.237535e+09


In [41]:
borrowings = self_borrow.merge(total_borrow, how='left', on='timestamp')
borrowings['selfBorrowUSD'] = borrowings['selfBorrowUSD'].fillna(0)
borrowings['selfBorrowShare'] = borrowings["selfBorrowUSD"] / borrowings["totalBorrowUSD"]
borrowings

,symbol_bor,timestamp,selfBorrowUSD,totalBorrowUSD,totalCollateralUSD,selfBorrowShare
0,cAAVE,2021-08-19,2489.868402,7.914928e+09,1.444673e+10,3.145788e-07
1,cAAVE,2021-08-20,2722.808655,8.025064e+09,1.487075e+10,3.392881e-07
2,cAAVE,2021-08-21,4610.444347,8.088935e+09,1.506277e+10,5.699693e-07
3,cAAVE,2021-08-22,4503.303663,8.091316e+09,1.503731e+10,5.565601e-07
4,cAAVE,2021-08-23,4627.488659,8.139425e+09,1.519342e+10,5.685277e-07
...,...,...,...,...,...,...
14872,cZRX,2023-07-27,52935.397436,5.696505e+08,1.259088e+09,9.292610e-05
14873,cZRX,2023-07-28,52933.375206,5.718996e+08,1.268614e+09,9.255711e-05
14874,cZRX,2023-07-29,52931.189305,5.725436e+08,1.268509e+09,9.244919e-05
14875,cZRX,2023-07-30,52933.307208,5.577766e+08,1.237535e+09,9.490055e-05


In [42]:
alt.Chart(borrowings[borrowings['symbol_bor'].isin(['cUSDC', 'cCOMP', 'cDAI', 'cETH', 'cZRX', 'cWBTC2'])]).mark_bar().encode(
    x='timestamp:T',
    y='selfBorrowShare',
    color='symbol_bor'
).interactive(bind_y = False)

alt.Chart(...)

Little historical view regarding the rewards:

- **June 15th, 2020**: Every Ethereum block, 0.50 COMP will be distributed across ETH, DAI, USDC, USDT, BAT, REP, WBTC and ZRX markets

- **June 27th, 2020**: Lower the number of distributed COMP to 0.44

- **August 31st, 2020**: Reduce COMP emissions by 20%

- **October 17th, 2020**: Add COMP Support

- **September 29th, 2021**: two different COMP distribution rates for each and every market - borrow-side rate and supply-side rate.

- **November 12th, 2021**: End cCOMP Borrow Rewards

- **March 27th, 2022**: the governance has voted to cut the existing rewards by 50%

- **June 29th, 2023**: Top-up COMP Rewards

## COMP Price

In [43]:
comp = pd.read_csv('../data/COMP_USD.csv', sep=';')

comp

,timeOpen,timeClose,timeHigh,timeLow,name,open,high,low,close,volume,marketCap,timestamp
0,2024-02-08T00:00:00.000Z,2024-02-08T23:59:59.999Z,2024-02-08T23:19:00.000Z,2024-02-08T12:53:00.000Z,2781,53.210016,54.087003,53.195422,53.991065,27736555.03,4.357963e+08,2024-02-08T23:59:59.999Z
1,2024-02-07T00:00:00.000Z,2024-02-07T23:59:59.999Z,2024-02-07T20:50:00.000Z,2024-02-07T08:14:00.000Z,2781,52.522981,53.555882,52.007126,53.210343,27746092.59,4.294929e+08,2024-02-07T23:59:59.999Z
2,2024-02-06T00:00:00.000Z,2024-02-06T23:59:59.999Z,2024-02-06T13:55:00.000Z,2024-02-06T14:16:00.000Z,2781,52.186594,53.102701,52.098893,52.523693,31031166.03,4.239104e+08,2024-02-06T23:59:59.999Z
3,2024-02-05T00:00:00.000Z,2024-02-05T23:59:59.999Z,2024-02-05T13:40:00.000Z,2024-02-05T01:16:00.000Z,2781,53.479430,53.512569,52.058446,52.185942,30687682.36,4.211833e+08,2024-02-05T23:59:59.999Z
4,2024-02-04T00:00:00.000Z,2024-02-04T23:59:59.999Z,2024-02-04T00:05:00.000Z,2024-02-04T22:43:00.000Z,2781,54.883572,54.900357,53.094205,53.497120,38982054.06,4.317643e+08,2024-02-04T23:59:59.999Z
...,...,...,...,...,...,...,...,...,...,...,...,...
1328,2020-06-20T00:00:00.000Z,2020-06-20T23:59:59.999Z,2020-06-20T23:05:23.000Z,2020-06-20T09:05:36.000Z,2781,224.669304,264.110007,197.410857,258.911624,2030188.91,6.631449e+08,2020-06-20T23:59:59.999Z
1329,2020-06-19T00:00:00.000Z,2020-06-19T23:59:59.999Z,2020-06-19T11:32:23.000Z,2020-06-19T00:00:20.000Z,2781,143.393405,241.837628,143.393405,224.545206,2524624.30,5.751229e+08,2020-06-19T23:59:59.999Z
1330,2020-06-18T00:00:00.000Z,2020-06-18T23:59:59.999Z,2020-06-18T13:08:08.000Z,2020-06-18T13:56:05.000Z,2781,64.267031,176.594731,61.271270,148.669784,1443324.86,0.000000e+00,2020-06-18T23:59:59.999Z
1331,2020-06-17T00:00:00.000Z,2020-06-17T23:59:59.999Z,2020-06-17T06:07:19.000Z,2020-06-17T11:32:18.000Z,2781,93.266296,113.041062,61.593316,64.635640,175376.44,0.000000e+00,2020-06-17T23:59:59.999Z


In [44]:
alt.Chart(comp).mark_line().encode(
    x='timestamp:T',
    y='close:Q'
).interactive(bind_y = False)

alt.Chart(...)

In [45]:
price = comp[comp['timestamp'] < '2023-07-31T23:59:59.999Z'].reset_index()[['timestamp', 'close']]
price['timestamp'] = price['timestamp'].apply(lambda date: datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%fZ'))

price['timestamp'] = price['timestamp'].dt.strftime('%Y-%m-%d')

price = price.rename(columns={'close': 'USD_price'})

price = price[::-1].reset_index(drop=True)

price

,timestamp,USD_price
0,2020-06-16,93.198607
1,2020-06-17,64.635640
2,2020-06-18,148.669784
3,2020-06-19,224.545206
4,2020-06-20,258.911624
...,...,...
1135,2023-07-26,70.692718
1136,2023-07-27,70.095282
1137,2023-07-28,72.094845
1138,2023-07-29,72.280781


In [46]:
price['price_variation (%)'] = (price['USD_price'] - price['USD_price'].shift(1)) / price['USD_price'].shift(1)
price

,timestamp,USD_price,price_variation (%)
0,2020-06-16,93.198607,NaN
1,2020-06-17,64.635640,-0.306474
2,2020-06-18,148.669784,1.300121
3,2020-06-19,224.545206,0.510362
4,2020-06-20,258.911624,0.153049
...,...,...,...
1135,2023-07-26,70.692718,0.133503
1136,2023-07-27,70.095282,-0.008451
1137,2023-07-28,72.094845,0.028526
1138,2023-07-29,72.280781,0.002579


In [47]:
alt.Chart(price).mark_line().encode(
    x='timestamp:T',
    y='price_variation (%):Q'
).interactive(bind_y = False)

alt.Chart(...)

# Fees 

In [48]:
markets = pd.read_csv('../data/balance_sheets/daily_markets.csv')

markets

,borrowRate,cash,collateralFactor,exchangeRate,interestRateModelAddress,name,reserves,supplyRate,symbol,cTokenAddress,...,underlyingPrice,underlyingSymbol,accrualBlockNumber,blockTimestamp,borrowIndex,reserveFactor,underlyingPriceUSD,underlyingDecimals,date,newPrice
0,0.039471,1.440748e+01,0.50,0.020000,0xbae04cbf96391086dc643e842b517734e214d698,Compound Augur,0.000000e+00,0.002306,cREP,0x158079ee67fce2f58472a96584a73c7ab9ac95c1,...,0.116547,REP,7715069,1557250486,1.000041,100000000000000000,20.004143,18,2019-05-07,NaN
1,0.016183,1.135850e+03,0.75,0.020000,0xc64c4cba055efa614ce01f4bad8a9f519c4f8fab,Compound USD Coin,6.000000e-06,0.001179,cUSDC,0x39aa39c021dfbae8fac545936693ac917d5e7563,...,0.005846,USDC,7716064,1557264374,1.000001,100000000000000000,1.000000,6,2019-05-07,NaN
2,0.012274,1.530022e+01,0.75,0.020000,0xc64c4cba055efa614ce01f4bad8a9f519c4f8fab,Compound Ether,4.704680e-06,0.000678,cETH,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5,...,1.000000,ETH,7716242,1557266749,1.000047,100000000000000000,171.043415,18,2019-05-07,NaN
3,0.033222,4.381321e+03,0.60,0.020000,0xbae04cbf96391086dc643e842b517734e214d698,Compound Basic Attention Token,4.561337e-04,0.001318,cBAT,0x6c8c6b02e7b2be14d4fa6022dfd6d75921d90e4e,...,0.001898,BAT,7712572,1557217208,1.000026,100000000000000000,0.335843,18,2019-05-07,NaN
4,0.054585,7.674348e+02,0.60,0.020000,0xbae04cbf96391086dc643e842b517734e214d698,Compound 0x,1.724746e-04,0.005664,cZRX,0xb3319f5d18bc0d84dd1b4825dcde5d5f7266d407,...,0.001559,ZRX,7711781,1557206685,1.000021,100000000000000000,0.276315,18,2019-05-07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21108,0.021737,1.359424e+04,0.70,0.020076,0xf2e5db36b0682f2cd6bc805c3a4236194e01f4d5,Compound Wrapped BTC,1.206402e+02,0.000134,cWBTC2,0xccf4429db6322d5c611ee964527d42e5d685dd6a,...,15.738445,WBTC,17816367,1690847207,1.086955,200000000000000000,29204.203950,8,2023-07-31,NaN
21109,0.026027,2.294171e+04,0.73,0.020626,0xd956188795ca6f4a74092ddca33e0ea4ca3a1395,Compound Aave Token,5.945144e+02,0.000523,cAAVE,0xe65cdb6479bac1e22340e4e755fae7e509ecd06c,...,0.035477,AAVE,17814861,1690829063,1.161843,250000000000000000,66.062500,18,2023-07-31,NaN
21110,0.050178,1.688357e+05,0.00,0.021450,0xa1046abfc2598f48c44fb320d281d3f3c0733c9a,Compound Dai,9.199748e+00,0.000000,cSAI,0xf5dce57282a584d2746faf1593d3121fcac444dc,...,0.005285,DAI,17691182,1690125815,1.388610,1000000000000000000,9.904040,18,2023-07-31,NaN
21111,0.037653,5.667166e+07,0.00,0.022459,0xfb564da37b41b2f6b6edcc3e56fbf523bd9f2012,Compound USDT,3.608744e+06,0.026228,cUSDT,0xf650c3d88d12db855b8bf7d11be6c55a4e07dcc9,...,0.000538,USDT,17816239,1690845659,1.203922,75000000000000000,1.000000,6,2023-07-31,NaN


In [49]:
fees = []
for symbol in markets['symbol'].unique():
    asset_fees = markets[markets['symbol'] == symbol]
    asset_fees['fee_variation (%)'] = (asset_fees['borrowRate'] - asset_fees['borrowRate'].shift(1)) / asset_fees['borrowRate'].shift(1)

    fees.append(asset_fees) 

fees = pd.concat(fees)[['symbol', 'fee_variation (%)', 'borrowRate', 'date']]
fees = fees.rename(columns={'date': 'timestamp'})

fees

/tmp/ipykernel_1836223/494316825.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_fees['fee_variation (%)'] = (asset_fees['borrowRate'] - asset_fees['borrowRate'].shift(1)) / asset_fees['borrowRate'].shift(1)
/tmp/ipykernel_1836223/494316825.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_fees['fee_variation (%)'] = (asset_fees['borrowRate'] - asset_fees['borrowRate'].shift(1)) / asset_fees['borrowRate'].shift(1)
/tmp/ipykernel_1836223/494316825.py:4: SettingWithCopyWarning: 
A value is t

,symbol,fee_variation (%),borrowRate,timestamp
0,cREP,NaN,0.039471,2019-05-07
6,cREP,0.0,0.039471,2019-05-08
12,cREP,0.0,0.039471,2019-05-09
18,cREP,0.0,0.039471,2019-05-10
24,cREP,0.0,0.039471,2019-05-11
...,...,...,...,...
21028,cFEI,0.0,0.174000,2023-07-27
21047,cFEI,0.0,0.174000,2023-07-28
21066,cFEI,0.0,0.174000,2023-07-29
21085,cFEI,0.0,0.174000,2023-07-30


In [50]:
alt.Chart(fees[fees['symbol'].isin(['cUSDC', 'cCOMP', 'cDAI', 'cETH', 'cZRX', 'cWBTC2'])]).mark_line().encode(
    x='timestamp:T',
    y='fee_variation (%)',
    color='symbol'
).interactive(bind_y = False)

alt.Chart(...)

# TVL

In [51]:
import json

json_file = 'daily_TVL.json'

with open(json_file, 'r') as f:
    data = json.load(f)

tvl = []
for entry in data['tvl']:
    tvl.append({'TVL': entry['totalLiquidityUSD']})

tvl = pd.DataFrame(tvl)
end_date = pd.to_datetime('2024-02-28')
start_date = end_date - pd.Timedelta(days=len(tvl) - 1)
tvl['timestamp'] = pd.date_range(start=start_date, end=end_date, freq='D')

tvl

,TVL,timestamp
0,9.546158e+03,2018-10-08
1,1.082613e+04,2018-10-09
2,1.250562e+04,2018-10-10
3,1.260091e+04,2018-10-11
4,1.307968e+04,2018-10-12
...,...,...
1965,2.578925e+09,2024-02-24
1966,2.665743e+09,2024-02-25
1967,2.726812e+09,2024-02-26
1968,2.854632e+09,2024-02-27


In [52]:
alt.Chart(tvl).mark_line().encode(
    x='timestamp:T',
    y='TVL:Q'
).interactive(bind_y = False)

alt.Chart(...)

In [77]:
import statsmodels.api as sm
import numpy as np


x = price[(price['timestamp'] < '2023-07-31') & (price['timestamp']> '2021-08-19')]['price_variation (%)']
bor = borrowings[(borrowings['timestamp'] < '2023-07-31') & (borrowings['timestamp']> '2021-08-19')]
z = tvl[(tvl['timestamp'] < '2023-07-31') & (tvl['timestamp']> '2021-08-19')]['TVL']
fee = fees[(fees['timestamp'] < '2023-07-31') & (fees['timestamp']> '2021-08-19')]

symbols = ['cUSDC', 'cCOMP', 'cETH', 'cDAI', 'cZRX', 'cWBTC2']

def get_reg(symbol):
    y = bor.groupby('symbol_bor').get_group(symbol)['selfBorrowShare']
    t = fee.groupby('symbol').get_group(symbol)['fee_variation (%)']
    model_xz = sm.OLS(y, sm.add_constant(np.column_stack((x, z)))).fit()
    print(model_xz.summary())


In [85]:
get_reg('cZRX')

                            OLS Regression Results                            
Dep. Variable:        selfBorrowShare   R-squared:                       0.671
Model:                            OLS   Adj. R-squared:                  0.670
Method:                 Least Squares   F-statistic:                     722.1
Date:                Fri, 01 Mar 2024   Prob (F-statistic):          1.45e-171
Time:                        19:15:18   Log-Likelihood:                 4389.3
No. Observations:                 710   AIC:                            -8773.
Df Residuals:                     707   BIC:                            -8759.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0004   3.18e-05    -12.436      0.0

In [86]:
get_reg('cUSDC')

                            OLS Regression Results                            
Dep. Variable:        selfBorrowShare   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     297.8
Date:                Fri, 01 Mar 2024   Prob (F-statistic):           1.54e-94
Time:                        19:15:44   Log-Likelihood:                 1227.4
No. Observations:                 710   AIC:                            -2449.
Df Residuals:                     707   BIC:                            -2435.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0740      0.003     27.119      0.0

In [87]:
get_reg('cWBTC2')

                            OLS Regression Results                            
Dep. Variable:        selfBorrowShare   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     1629.
Date:                Fri, 01 Mar 2024   Prob (F-statistic):          1.95e-265
Time:                        19:16:02   Log-Likelihood:                 3086.9
No. Observations:                 710   AIC:                            -6168.
Df Residuals:                     707   BIC:                            -6154.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0031      0.000    -15.591      0.0

In [89]:
get_reg('cDAI' )

                            OLS Regression Results                            
Dep. Variable:        selfBorrowShare   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.389
Method:                 Least Squares   F-statistic:                     227.0
Date:                Fri, 01 Mar 2024   Prob (F-statistic):           7.00e-77
Time:                        19:16:40   Log-Likelihood:                 870.17
No. Observations:                 710   AIC:                            -1734.
Df Residuals:                     707   BIC:                            -1721.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1444      0.005     31.975      0.0

In [90]:
get_reg('cCOMP')

                            OLS Regression Results                            
Dep. Variable:        selfBorrowShare   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     188.1
Date:                Fri, 01 Mar 2024   Prob (F-statistic):           3.07e-66
Time:                        19:17:00   Log-Likelihood:                 4574.3
No. Observations:                 710   AIC:                            -9143.
Df Residuals:                     707   BIC:                            -9129.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0002   2.45e-05     -7.578      0.0

In [91]:
get_reg('cETH')

                            OLS Regression Results                            
Dep. Variable:        selfBorrowShare   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     2363.
Date:                Fri, 01 Mar 2024   Prob (F-statistic):          8.54e-314
Time:                        19:17:29   Log-Likelihood:                 2956.3
No. Observations:                 710   AIC:                            -5907.
Df Residuals:                     707   BIC:                            -5893.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0025      0.000    -10.651      0.0